# 2.7 학습 및 검증 실시

- SSD 학습과 검증의 실시를 수행합니다. 로컬 머신으로 동작을 확인한 뒤, AWS의 GPU 머신으로 계산합니다.
- p2.xlarge에서 약 6시간 걸립니다.


# 학습 목표

1.	SSD 학습을 구현할 수 있다

# 사전 준비

- AWS EC2의 GPU 인스턴스를 사용합니다
- "utils" 폴더의 ssd_model.py를 실행합니다

In [10]:
# 패키지 import
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [11]:
# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)

사용 중인 장치: cuda:0


# Dataset과 DataLoader를 작성한다

In [13]:
from utils.ssd_model_modified import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn

# 파일 경로 리스트를 취득
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Dataset 작성
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR) 색의 평균값
input_size = 300  # 이미지의 input 크기를 300×300으로 설정

class_distribution = {
 'person': 0.4335293238327769,
 'train': 0.017539488763765012,
 'chair': 0.07613732622452539,
 'diningtable': 0.019931237231551147,
 'boat': 0.026383975285265832,
 'bus': 0.017066121879515672,
 'car': 0.06208580397628183,
 'horse': 0.020005979371169467,
 'tvmonitor': 0.02224824355971897,
 'bottle': 0.03889082664806418,
 'pottedplant': 0.029946683940405602,
 'cat': 0.03181523743086352,
 'bird': 0.03166575315162689,
 'cow': 0.01920872988190742,
 'dog': 0.03981264637002342,
 'aeroplane': 0.024963874632517813,
 'bicycle': 0.02085305695351039,
 'sofa': 0.020952713139668146,
 'motorbike': 0.019956151278090587,
 'sheep': 0.027006826448751808
}

class_sample_counts = {
 'person': 17401,
 'train': 704,
 'chair': 3056,
 'diningtable': 800,
 'boat': 1059,
 'bus': 685,
 'car': 2492,
 'horse': 803,
 'tvmonitor': 893,
 'bottle': 1561,
 'pottedplant': 1202,
 'cat': 1277,
 'bird': 1271,
 'cow': 771,
 'dog': 1598,
 'aeroplane': 1002,
 'bicycle': 837,
 'sofa': 841,
 'motorbike': 801,
 'sheep': 1084
}
train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoader를 작성
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 사전 오브젝트로 정리
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# 네트워크 모델을 작성한다

In [14]:
from utils.hj_ssd_model_modified import SSD

# SSD300 설정
ssd_cfg = {
            'num_classes': 21,
            'input_size': 300,  # 이미지의 입력 크기\n",
            'bbox_aspect_num': [4, 6, 6, 6,6, 4, 4, 4],  # 출력할 DBox의 화면비의 종류\n",
             'feature_maps': [38, 19,19, 10, 5, 3,3, 1],  # 각 source의 이미지 크기\n",
                  'steps': [8, 16, 16, 32, 64, 100, 100, 300],
                'min_sizes': [30, 60, 60, 111, 162, 213,213 , 264],  # DBOX의 크기(최소)\n",
             'max_sizes': [60, 111, 111, 162, 213, 264, 264, 315],  # DBOX의 크기(최대)\n",
                 'aspect_ratios': [[2], [2,3], [2, 3], [2, 3], [2, 3], [2], [2], [2]]}

# SSD 네트워크 모델
net = SSD(phase="train", cfg=ssd_cfg)

# SSD의 초기 가중치를 설정
# ssd의 vgg 부분에 가중치를 로드한다
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssd의 기타 네트워크의 가중치는 He의 초기치로 초기화
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # 바이어스 항이 있는 경우
            nn.init.constant_(m.bias, 0.0)


# He의 초기치를 적용
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPU를 사용할 수 있는지 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("사용 중인 장치:", device)

print('네트워크 설정 완료: 학습된 가중치를 로드했습니다')


사용 중인 장치: cuda:0
네트워크 설정 완료: 학습된 가중치를 로드했습니다


# 손실함수 및 최적화 기법의 설정

In [15]:
from utils.ssd_model import MultiBoxLoss

# 손실함수의 설정
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 최적화 기법의 설정
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 학습 및 검증을 실시

In [16]:
import tqdm

# 모델을 학습시키는 함수 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPU를 사용할 수 있는지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 중인 장치:", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면, 고속화시킨다
    torch.backends.cudnn.benchmark = True

    # 반복자의 카운터 설정
    iteration = 1
    epoch_train_loss = 0.0  # epoch의 손실합
    epoch_val_loss = 0.0  # epoch의 손실합
    logs = []

    # epoch 루프
    for epoch in range(num_epochs+1):

        # 시작 시간을 저장
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증을 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련모드로
                print('(train)')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # 모델을 검증모드로
                    print('-------------')
                    print('(val)')
                else:
                    # 검증은 10번에 1번만 실시
                    continue

            # 데이터 로더에서 minibatch씩 꺼내 루프
            iterator = tqdm.tqdm(dataloaders_dict[phase])
            print(len(dataloaders_dict[phase]))
            #for images, targets in dataloaders_dict[phase]:
            for images, targets in iterator:
                
                # GPU를 사용할 수 있으면, GPU에 데이터를 보낸다
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # 리스트의 각 요소의 텐서를 GPU로

                # optimizer를 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    # 순전파(forward) 계산
                    outputs = net(images)

                    # 손실 계산
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 훈련시에는 역전파(Backpropagation)
                    if phase == 'train':
                        loss.backward()  # 경사 계산

                        # 경사가 너무 커지면 계산이 불안정해지므로, clip에서 최대라도 경사 2.0에 고정
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # 파라미터 갱신

                        #if (iteration % 10 == 0):  # 10iter에 한 번, loss를 표시
                        t_iter_finish = time.time()
                        duration = t_iter_finish - t_iter_start
                        iterator.set_description('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                        iteration, loss.item(), duration))
                        t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 검증시
                    else:
                        epoch_val_loss += loss.item()

        # epoch의 phase 당 loss와 정답률
        t_epoch_finish = time.time()
        epoch_train_loss/=len(dataloaders_dict[phase])
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # 로그를 저장
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output_fm_8.csv")

        epoch_train_loss = 0.0  # epoch의 손실합
        epoch_val_loss = 0.0  # epoch의 손실합

        # 네트워크를 저장한다
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_fm_8' +
                       str(epoch+1) + '.pth')


In [17]:
# 학습 및 검증 실시
num_epochs= 100  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 중인 장치: cuda:0
-------------
Epoch 1/100
-------------
(train)


  0%|          | 0/179 [00:00<?, ?it/s]

179


/home/iuser/workspace/objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
반복 179 || Loss: 8.3002 || 10iter: 0.3842 sec.: 100%|██████████| 179/179 [02:12<00:00,  1.35it/s]


epoch 1 || Epoch_TRAIN_Loss:9.7504 ||Epoch_VAL_Loss:0.0000
timer:  132.6197 sec.
-------------
Epoch 2/100
-------------
(train)


...

epoch 100 || Epoch_TRAIN_Loss:2.9369 ||Epoch_VAL_Loss:683.2381
timer:  169.1683 sec.
-------------
Epoch 101/100
-------------
(train)


  0%|          | 0/179 [00:00<?, ?it/s]

179


반복 18079 || Loss: 3.4510 || 10iter: 0.2954 sec.: 100%|██████████| 179/179 [01:58<00:00,  1.51it/s]

epoch 101 || Epoch_TRAIN_Loss:2.8967 ||Epoch_VAL_Loss:0.0000
timer:  118.9347 sec.
